# Team AVYULAUNCH

# 11/13/2021
### Combine weather and avalanche data!

In [13]:
import pandas as pd
import numpy as np

In [14]:
def get_dataframe():
    pd.set_option('display.max_rows', 20)

    # Weather from NOAA Database
    df_weather = pd.read_csv('weather with provo.csv')

    # Just keeping core features
    df_weather = df_weather[['NAME', 'DATE', 'PRCP', 'SNWD', 'TMAX', 'TMIN']]

    # Get month dummy variables
    df_weather['DATE'] = pd.to_datetime(df_weather['DATE'])
    df_weather['MONTH'] = pd.DatetimeIndex(df_weather['DATE']).month
    df_weather['MONTH'] = df_weather['MONTH'].astype(str)
    df_weather = pd.get_dummies(df_weather, columns=['MONTH'])


    # Create Region variable so we can merge with Avalanche dataset
    def assign_region(name):
        if name == "BEN LOMOND PEAK, UT US": return "Ogden"
        if name == "ALTA, UT US": return "Salt Lake"
        if name == "BEN LOMOND TRAIL, UT US": return "Ogden"
        if name == "MONTE CRISTO, UT US": return "Logan"
        if name == "BUES CANYON UTAH, UT US": return "Ogden"
        if name == "RAY S VALLEY UTAH, UT US": return "Uintas"
        if name == "SNOWBIRD, UT US": return "Salt Lake"
        if name == "PROVO BYU, UT US": return "Provo"

    # Create snow difference by Weather station NAME
    df_list = []
    by_location = df_weather.groupby('NAME')
    for name, group in by_location:
        group['Region'] = assign_region(name)

        # Snow depth of the day before minus the current day
        group['snow_diff_day'] = group['SNWD'] - group['SNWD'].shift(1)
        # Change in snow over the last week
        group['snow_diff_week'] = group['SNWD'] - group['SNWD'].shift(7)

        # Binary saying if we got snow from the day before or not
        group['got_snow'] = (group['snow_diff_day'] > 0).astype(int) 
        df_list.append(group)
    df_weather = pd.concat(df_list)

    # Create indicator for if it was below freezing at any point that day
    df_weather['min_below_freezing'] = (df_weather['TMIN'] < 32).astype(int)
    # Create indicator for if it was above freezing at any point that day
    df_weather['max_above_freezing'] = (df_weather['TMAX'] > 32).astype(int)

    # min * max means:
    # 1 if min below freezing and max above freezing
    # 0 otherwise
    # This is potentially significant if we cross the freezing point of water in a day
    df_weather['min*max'] = df_weather['min_below_freezing'] * df_weather['max_above_freezing']


    
    # Clean up avalanche data
    df_avalanche = pd.read_csv('final_avalanche.csv')
    # Delete rows without date or region
    df_avalanche = df_avalanche[['Date', 'Region']].dropna()
    # Convert to date time
    df_avalanche['DATE'] = pd.to_datetime(df_avalanche['Date'])
    df_avalanche['Avalanche'] = 1
    
    # Combine avalanche and weather on region and date
    df_combined = pd.merge(df_weather, df_avalanche,  how='left', on=['Region', 'DATE'])
    
    # Create variable of summed up avalanches per day by Weather Station NAME
    summed = df_combined.groupby(['NAME', 'DATE'])['Avalanche'].agg('sum').reset_index()
    summed['avalanche_sum'] = summed['Avalanche']

    # Add new column back onto original dataframe
    reassembled = pd.merge(summed, df_combined, how='left', on=['NAME', 'DATE'])
    reassembled = reassembled.drop_duplicates()

    # Create new column: binary indicator if there was an avalanche that day or not
    reassembled['avalanche_binary'] = reassembled['avalanche_sum'] > 0

    # Final clean up: drop unnecessary columns
    df_final = reassembled.drop(['Avalanche_x', 'Avalanche_y', 'Date', 'NAME'], axis=1)

    # Create region dummy variables
    df_final = pd.get_dummies(df_final, columns=['Region'], drop_first=True)

    # Drop rows with nan variables
    df_final = df_final.dropna()

    df_final = df_final[df_final['DATE'] > '2010-01-01']

    df_final.to_csv("FINAL_DF.csv")
    
    return df_final

In [12]:
df = get_dataframe()
print(df)

Index(['NAME', 'DATE', 'PRCP', 'SNWD', 'TMAX', 'TMIN', 'MONTH_1', 'MONTH_10',
       'MONTH_11', 'MONTH_12', 'MONTH_2', 'MONTH_3', 'MONTH_4', 'MONTH_5',
       'MONTH_6', 'MONTH_7', 'MONTH_8', 'MONTH_9', 'Region', 'snow_diff_day',
       'snow_diff_week', 'got_snow', 'min_below_freezing',
       'max_above_freezing', 'min*max'],
      dtype='object')
Index(['Date', 'Region', 'DATE', 'Avalanche'], dtype='object')
Index(['DATE', 'avalanche_sum', 'PRCP', 'SNWD', 'TMAX', 'TMIN', 'MONTH_1',
       'MONTH_10', 'MONTH_11', 'MONTH_12', 'MONTH_2', 'MONTH_3', 'MONTH_4',
       'MONTH_5', 'MONTH_6', 'MONTH_7', 'MONTH_8', 'MONTH_9', 'snow_diff_day',
       'snow_diff_week', 'got_snow', 'min_below_freezing',
       'max_above_freezing', 'min*max', 'avalanche_binary', 'Region_Ogden',
       'Region_Provo', 'Region_Salt Lake', 'Region_Uintas'],
      dtype='object')
            DATE  avalanche_sum  PRCP  SNWD  TMAX  TMIN  MONTH_1  MONTH_10  \
3519  2010-01-03            1.0  0.00  38.0  31.0  11.0   